In [1]:
import os
import openai
from env_config import Config

openai.api_key = Config.OPENAI_API_KEY

In [11]:
import os
import openai
from env_config import Config

openai.api_key = Config.OPENAI_API_KEY

message = "tell me seomthing about the grand canyan"
# for i in range(len(message)):

for token in openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": message}
        ],
        stream = True,
        n = 1
        ):
    print(token["choices"][0]["delta"].get("content", ""), end="")

# print(resp)

The Grand Canyon is a steep-sided canyon carved by the Colorado River in the state of Arizona, USA. It is approximately 277 miles long, up to 18 miles wide, and over a mile deep. The Grand Canyon is known for its stunning vistas, unique geological features, and rich cultural history. It is also home to a diverse range of plants and animals, many of which are found nowhere else on earth. The Grand Canyon is a popular tourist destination, attracting millions of visitors every year who come to hike, camp, and explore this remarkable natural wonder.

In [7]:
system_message = "You are a PaulGrahamGPT \n Below is a informational summary about Paul Graham life \n"

file_path = r"C:\Users\DELL\Documents\python_projects\llm_tech\data\paul_graham\paul_graham_essay.txt"

with open(file_path) as f:
    text = f.read()
    system_message += text
    system_message = system_message.split('Notes')[0]

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": "When did Paul Graham's mother has the stroke? And what happened after that?"}
    ],
    )

print(completion)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Paul Graham's mother had a stroke in the summer of 2012. As a result of the stroke, her balance was destroyed, and she had to be placed in a nursing home. However, Paul and his sister were determined to help her get out and return to her house. Paul frequently flew to Oregon to visit her, and during these visits, he had a lot of time to reflect.\n\nUnfortunately, his mother's cancer had returned. The cancer was the cause of the blood clot that led to her stroke. Sadly, she passed away on January 15, 2014. This was a difficult time for Paul and his family, as losing a loved one is always painful.\n\nFollowing his mother's passing, Paul decided to step back from his work at Y Combinator, the startup accelerator he co-founded. He wanted to take some time for himself and explore new interests. He began painting, which had always been a passion of his, and started writing essays ag

In [2]:
import openai
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )
    print(response)
    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = get_current_weather(
            location=message.get("location"),
            unit=message.get("unit"),
        )
        print(function_response)

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather like in boston?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

{
  "id": "chatcmpl-7sV0Qftfn2TtsZIqLxyULi83K30tA",
  "object": "chat.completion",
  "created": 1693223254,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  }
}
{"location": null, "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}
{
  "id": "chatcmpl-7sV0StBINbwpZHJOqIUJ2F9VGFr4Y",
  "object": "chat.completion",
  "created": 1693223256,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston, MA is sunny and windy with a temperature of 72 degrees."
      },
      "finish_reason

In [2]:
from serpapi import GoogleSearch

params = {
      "engine": "google",
      "q": "weather in delhi today",
      "api_key": Config.SERP_API_KEY
    }

search = GoogleSearch(params)
results = search.get_dict()
print(results.keys())

# organic_results = results["answer_box"]
# print(organic_results.keys())

# organic_results[:2]

NameError: name 'Config' is not defined

In [ ]:
import openai
import json
from serpapi import GoogleSearch

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def search_google(query):
    params = {
      "engine": "google",
      "q": query,
      "api_key": Config.SERP_API_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results["knowledge_graph"]

    return organic_results

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What is the weather in the Delhi today?"}],
        functions=[
            {
                "name": "search_google",
                "description": "Use this function to Search on google to provide accurate answers for current affairs",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "query to search on google search engine",
                        },
                    },
                    "required": ["query"],
                },
            }
        ],
        function_call="auto",
    )
    print(response)
    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        query = eval(message["function_call"]['arguments'])

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = search_google(query['query'])
        print(function_response[:2])

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather forcast for Delhi today?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response[:2],
                },
            ],
        )
        return second_response

print(run_conversation())

In [12]:
# json formatting with functions

import openai
import json

def to_json(
        name, age, powers, history
        # **kwargs
        ):
    _json = {
        'name': name,
        'age': int(age),
        'powers': powers.split(', '),
        'history': history
    }
    _json = json.dumps(_json, indent=4)

    return _json

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "user", "content": "Generate a RPG game fake character profile and return in json"}],
        functions=[
            {
                "name": "to_json",
                "description": "use this function to covert character data to json format",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "description": {
                            "type": "string",
                            "description": "description of the character: A quick and nimble fighter.",
                        },
                        "name": {
                            "type": "string",
                            "description": "name of the character",
                        },
                        "age": {
                            "type": "string",
                            "description": "age of the character",
                        },
                        "weapons": {
                            "type": "string",
                            "description": 'one of ["sword", "axe", "mace", "spear", "bow", "crossbow"]',
                        },
                        "class": {
                            "type": "string",
                            "description": "",
                        },
                        "mantra": {
                            "type": "string",
                            "description": "",
                        },
                        "strength": {
                            "type": "string",
                            "description": "a number",
                        },
                        "items": {
                            "type": "string",
                            "description": "3 items",
                        },
                    },
                    "required": [],
                },
            },
        ],
        function_call="auto",
    )
    print(response)
    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        # function_name = message["function_call"]["name"]
        args = eval(message["function_call"]['arguments'])

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = to_json(**args)
        print(function_response)

        # Step 4, send model the info on the function call and function response
        # second_response = openai.ChatCompletion.create(
        #     model="gpt-3.5-turbo-0613",
        #     messages=[
        #         {"role": "user", "content": "Generate a RPG game fake character profile and return in json"},
        #         message,
        #         {
        #             "role": "function",
        #             "name": function_name,
        #             "content": function_response,
        #         },
        #     ],
        # )
        # return second_response

run_conversation()

{
  "id": "chatcmpl-7dEHBWqSmcDN2rEkrjirl7ZodPYj1",
  "object": "chat.completion",
  "created": 1689584025,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "to_json",
          "arguments": "{\n  \"name\": \"Aria\",\n  \"age\": \"25\",\n  \"powers\": \"Fire Manipulation, Telekinesis, Invisibility\",\n  \"history\": \"Aria grew up in a small village surrounded by mountains. At a young age, she discovered her ability to control fire and soon mastered the art of fire manipulation. She then developed telekinetic powers, allowing her to move objects with her mind. As she further honed her skills, she learned how to become invisible, blending seamlessly into her surroundings. Aria became a powerful and elusive warrior, using her unique abilities to protect her village from external threats.\"\n}"
        }
      },
      "finish_reason": "function_